In [1]:
import cv2
import numpy as np
import time

# Start processing timer
start = time.time()

# Load in the sample image
sample = cv2.imread('../img/sample-2.png')

# Convert to grayscale
grayscale = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY)

The binary thresholding is very unstable and not suitable.

In [2]:
# Binary thresholding
# Apply binary thresholding
# Parameters:
#   src: input grayscale image
#   thresh: threshold value
#   maxval: maximum value to use with the binary thresholding type
#   type: thresholding type (cv2.THRESH_BINARY, cv2.THRESH_BINARY_INV, etc.)
# Returns:
#   ret: the threshold value (not used in this example)
#   binary_thresholded_sample: the binary thresholded image
# ret, binary_thresholded_sample = cv2.threshold(grayscale, 90, 255, cv2.THRESH_BINARY)

We'll instead use an edge detection algorthim, such as Canny.

In [3]:
# Apply Canny edge detection
# Parameters:
#   image: input grayscale image
#   threshold1: first threshold for the hysteresis procedure
#   threshold2: second threshold for the hysteresis procedure
# Returns:
#   edges: output edge map; single channels 8-bit image, which has the same size as image
edges = cv2.Canny(grayscale, 100, 200)

# Find contours
# Parameters:
#   image: input 8-bit single-channel image
#   mode: contour retrieval mode (cv2.RETR_EXTERNAL: retrieves only the extreme outer contours)
#   method: contour approximation method (cv2.CHAIN_APPROX_SIMPLE: compresses horizontal, vertical, and diagonal segments and leaves only their end points)
# Returns:
#   contours: a Python list of all the contours in the image. Each individual contour is a Numpy array of (x, y) coordinates of boundary points of the object
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
contour_image = np.zeros_like(grayscale)
cv2.drawContours(contour_image, contours, -1, (255, 255, 255), 2)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [4]:
# Apply Hough Circle Transform
# Parameters:
#   image: input image
#   method: detection method (cv2.HOUGH_GRADIENT)
#   dp: inverse ratio of the accumulator resolution to the image resolution (1)
#   minDist: minimum distance between the centers of the detected circles (10)
#   param1: first method-specific parameter (canny edge detection threshold)
#   param2: second method-specific parameter (accumulator threshold for the circle centers)
#   minRadius: minimum circle radius (0)
#   maxRadius: maximum circle radius (0)
circles = cv2.HoughCircles(contour_image, cv2.HOUGH_GRADIENT, dp=1, minDist=10,
                            param1=50, param2=30, minRadius=0, maxRadius=0)

# We'll make a copy of the original image, for annotating with circle(s)
annotated_sample = sample

# If circles are found, draw them
if circles is not None:
    circles = np.uint16(np.around(circles))
    for circle in circles[0, :]:
        center = (circle[0], circle[1])
        radius = circle[2]
        # Draw the circle outline
        cv2.circle(sample, center, radius, (0, 255, 0), 2)

In [5]:
# Stop the timer and calculate the processing time
stop = time.time()
processing_time = stop - start

# Display the original image
cv2.imshow('Original Image', sample)

# Display the grayscale image
cv2.imshow('Grayscale Image', grayscale)

cv2.imshow("Contour Image", contour_image)


# Display the image with detected circles
cv2.imshow('Detected Circles', annotated_sample)
cv2.waitKey(0)
cv2.destroyAllWindows()